# 索引 / 查询 JSON 文档
## 将 JSON 文档添加到索引中

In [1]:
import redis
from redis.commands.json.path import Path
import redis.commands.search.aggregation as aggregations
import redis.commands.search.reducers as reducers
from redis.commands.search.field import TextField, NumericField, TagField
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
from redis.commands.search.query import NumericFilter, Query


r = redis.Redis(host='localhost', port=6379)
user1 = {
    "user":{
        "name": "Paul John",
        "email": "paul.john@example.com",
        "age": 42,
        "city": "London"
    }
}
user2 = {
    "user":{
        "name": "Eden Zamir",
        "email": "eden.zamir@example.com",
        "age": 29,
        "city": "Tel Aviv"
    }
}
user3 = {
    "user":{
        "name": "Paul Zamir",
        "email": "paul.zamir@example.com",
        "age": 35,
        "city": "Tel Aviv"
    }
}

user4 = {
    "user":{
        "name": "Sarah Zamir",
        "email": "sarah.zamir@example.com",
        "age": 30,
        "city": "Paris"
    }
}
r.json().set("user:1", Path.root_path(), user1)
r.json().set("user:2", Path.root_path(), user2)
r.json().set("user:3", Path.root_path(), user3)
r.json().set("user:4", Path.root_path(), user4)

schema = (TextField("$.user.name", as_name="name"),TagField("$.user.city", as_name="city"), NumericField("$.user.age", as_name="age"))
r.ft().create_index(schema, definition=IndexDefinition(prefix=["user:"], index_type=IndexType.JSON))

b'OK'

## 搜索(Searching)

### 简单搜索(Simple search)

In [2]:
r.ft().search("Paul")

Result{2 total, docs: [Document {'id': 'user:1', 'payload': None, 'json': '{"user":{"name":"Paul John","email":"paul.john@example.com","age":42,"city":"London"}}'}, Document {'id': 'user:3', 'payload': None, 'json': '{"user":{"name":"Paul Zamir","email":"paul.zamir@example.com","age":35,"city":"Tel Aviv"}}'}]}

### 过滤搜索结果(Filtering search results)

In [3]:
q1 = Query("Paul").add_filter(NumericFilter("age", 30, 40))
r.ft().search(q1)

Result{1 total, docs: [Document {'id': 'user:3', 'payload': None, 'json': '{"user":{"name":"Paul Zamir","email":"paul.zamir@example.com","age":35,"city":"Tel Aviv"}}'}]}

### 分页和排序搜索结果(Paginating and Ordering search Results)

In [4]:
# Search for all users, returning 2 users at a time and sorting by age in descending order
offset = 0
num = 2
q = Query("*").paging(offset, num).sort_by("age", asc=False) # pass asc=True to sort in ascending order
r.ft().search(q)

Result{4 total, docs: [Document {'id': 'user:1', 'payload': None, 'age': '42', 'json': '{"user":{"name":"Paul John","email":"paul.john@example.com","age":42,"city":"London"}}'}, Document {'id': 'user:3', 'payload': None, 'age': '35', 'json': '{"user":{"name":"Paul Zamir","email":"paul.zamir@example.com","age":35,"city":"Tel Aviv"}}'}]}

### 计算项目的总数

In [5]:
q = Query("*").paging(0, 0)
r.ft().search(q).total

4

### 使用 JSON Path 表达式进行映射

In [6]:
r.ft().search(Query("Paul").return_field("$.user.city", as_field="city")).docs

[Document {'id': 'user:1', 'payload': None, 'city': 'London'},
 Document {'id': 'user:3', 'payload': None, 'city': 'Tel Aviv'}]

## 聚合(Aggregation)

In [7]:
req = aggregations.AggregateRequest("Paul").sort_by("@age")
r.ft().aggregate(req).rows

[[b'age', b'35'], [b'age', b'42']]

### 计算项目的总数

In [8]:
# `group_by` 期望一个字符串或字符串列表，以便在对每个组应用聚合函数之前对结果进行分组。
# 在此处传递一个空列表的效果相当于 `GROUPBY 0`，它将聚合函数应用于整个结果集。
req = aggregations.AggregateRequest("*").group_by([], reducers.count().alias("total"))
r.ft().aggregate(req).rows

[[b'total', b'4']]